https://github.com/giswqs/geemap/blob/master/examples/notebooks/31_unsupervised_classification.ipynb

In [1]:
import ee, geemap

In [2]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWjUNTZMpwtQhVcaOuR0to_JrC-ErBZv1y9WkQ7g4as3JkR6-xmDjWo



Successfully saved authorization token.


In [3]:
ee.Initialize()

## Create an interactive map

In [4]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Add SRER 2017 Surface Directional Reflectance

In [5]:
# Specify center location of SRER
geo = ee.Geometry.Point([-110.83549, 31.91068])

# Read in Surface Directional Reflectance (SDR) Images and filter by date and bounds 
SRER_SDR2017 = ee.ImageCollection("projects/neon/DP3-30006-001_SDR").filterDate('2017-01-01', '2017-12-31').filterBounds(geo).first();

# Mask layers to only show values > 0 (this hides the no data values of -9999) 
SRER_SDR2017mask = SRER_SDR2017.updateMask(SRER_SDR2017.gte(0.0000))

# Set the visualization parameters so contrast is maximized, and set display to show RGB bands 
visParams = {'min':2,'max':20,'gamma':0.9,'bands':['band053','band035','band019']};

Map.addLayer(SRER_SDR2017mask, visParams, 'SRER 2017');

Map.setCenter(-110.83549, 31.91068, 10);

## Remove water vapor absorption bands!

## Look at the image properties

props = geemap.image_props(SRER_SDR2017)
props.getInfo()

In [8]:
props.get('IMAGE_DATE').getInfo()

'2017-08-24'

## Make training dataset
There are several ways you can create a region for generating the training dataset.

Draw a shape (e.g., rectangle) on the map and the use region = Map.user_roi
Define a geometry, such as region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
Create a buffer zone around a point, such as region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)
If you don't define a region, it will use the image footprint by default

In [10]:
# Make the training dataset.
training = SRER_SDR2017.sample(**{
#     'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=106832.0, center=[31.91068, -110.83549], controls=(WidgetControl(options=['position', 'transparent_…

## Train the clusterer

In [15]:
# Instantiate the clusterer and train it.
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

## Classify the image

In [16]:
# Cluster the input using the trained clusterer.
result = SRER_SDR2017.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=426800.0, center=[31.801725465871957, -110.78596115112306], controls=(WidgetControl(options=['posit…

## Label the clusters

In [20]:
legend_keys = ['One', 'Two', 'Three', 'Four']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072']

# Reclassify the map
result = result.remap([0, 1, 2, 3], [1, 2, 3, 4])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

Map(bottom=1706013.0, center=[31.82265845614348, -110.93912601470949], controls=(WidgetControl(options=['posit…

In [17]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [ ]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')